In [1]:
#!pip install nltk
import networkx as nx
import pandas as pd
import nltk

In [2]:
class GraphTraversal:
    def createGraph(self, sentences):
        #DG=nx.DiGraph()
        DG=nx.MultiDiGraph()
        tokens = nltk.word_tokenize(sentences)
        token_count = len(tokens)
        for i in range(token_count):
            if i == 0:
                continue
            DG.add_edges_from([(tokens[i-1], tokens[i])], weight=1)
        return DG


    def getMCS0(self, G_source, G_new):
        """
        Creator: Bonson
        Return the MCS of the G_new graph that is present in the G_source graph
        """
        order =  nx.topological_sort(G_new)
        print("##### topological sort #####")
        print(order)

        #objSubGraph = nx.DiGraph()
        objSubGraph = nx.MultiDiGraph()
        for i in range(len(order)-1):
            if G_source.nodes().__contains__(order[i]) and G_source.nodes().__contains__(order[i+1]):
                print("Contains Nodes {0} -> {1} ".format(order[i], order[i+1]))
                objSubGraph.add_node(order[i])
                objSubGraph.add_node(order[i+1])
                objSubGraph.add_edge(order[i], order[i+1])
            else:
                print("Does Not Contains Nodes {0} -> {1} ".format(order[i], order[i+1]))
                continue

    def getMCS1(self, G_source, G_new):
        matching_graph=nx.Graph()
        #matching_graph=nx.MultiDiGraph()
        for n1,n2,attr in G_new.edges(data=True):
            if G_source.has_edge(n1,n2) :
                matching_graph.add_edge(n1,n2,weight=1)
        #graphs = list(nx.connected_component_subgraphs(matching_graph))
        graphs = [matching_graph.subgraph(c) for c in nx.connected_components(matching_graph)]
        mcs_length = 0
        #mcs_graph = nx.Graph()
        mcs_graph = nx.MultiDiGraph()
        for i, graph in enumerate(graphs):
            if len(graph.nodes()) > mcs_length:
                mcs_length = len(graph.nodes())
                mcs_graph = graph
        return mcs_graph

    def getMCS2(self, g1, g2):
        matching_graph=nx.Graph()
        #matching_graph=nx.MultiDiGraph()
        for n1,n2 in g2.edges():
            if g1.has_edge(n1, n2):
                matching_graph.add_edge(n1, n2)

        components = nx.connected_components(matching_graph)
        largest_component = max(components, key=len)
        #print(type(largest_component))
        #return nx.induced_subgraph(matching_graph, largest_component)
        return nx.induced_subgraph(g1, largest_component)

In [3]:
obj_graph_traversal = GraphTraversal()
SourceSentences = "scene filter_size filter_color count scene filter_size filter_material count less_than . "
SourceGraph = obj_graph_traversal.createGraph(SourceSentences)

TestSentence_1 = "scene filter_size filter_color filter_material filter_shape count scene filter_size filter_color filter_material filter_shape count greater_than . "    #This DOES NOT Work
TestGraph = obj_graph_traversal.createGraph(TestSentence_1)

res = obj_graph_traversal.getMCS2(SourceGraph, TestGraph)

print(res.edges)

[('scene', 'filter_size', 0), ('scene', 'filter_size', 1), ('filter_size', 'filter_color', 0), ('filter_color', 'count', 0), ('count', 'scene', 0)]


In [4]:
def two_program(SourceSentences, TestSentence_1):
    obj_graph_traversal = GraphTraversal()
    SourceGraph = obj_graph_traversal.createGraph(SourceSentences)
    TestGraph = obj_graph_traversal.createGraph(TestSentence_1)
    res = obj_graph_traversal.getMCS2(SourceGraph, TestGraph)
    #print(res.edges)
    return res.edges

In [5]:
def double_edges(SourceSentences):
    obj_graph_traversal = GraphTraversal()
    SourceGraph = obj_graph_traversal.createGraph(SourceSentences)
    #TestGraph = obj_graph_traversal.createGraph(TestSentence_1)
    #res = obj_graph_traversal.getMCS2(SourceGraph, TestGraph)
    #print(res.edges)
    return SourceGraph.edges

In [7]:
import json
def import_question_json():
    with open('InputData/questions.json') as json_file:
        data_question = json.load(json_file)
    
    with open('OutputData/questions.json','w') as f:
        data_question_json = data_question['questions']
        json.dump(data_question_json,f)
    return data_question

#import_question_json()

In [8]:
def import_scene_json():
    with open('InputData/scenes.json') as json_file:
        data_scene = json.load(json_file)
    with open('OutputData/scenes.json','w') as f:
        data_scene_json = data_scene['scenes']
        json.dump(data_scene_json,f) 
    return data_scene

#import_scene_json()

In [9]:
def generate_txt():
    program_list = []
    question_list = []
    answer_list = []
    data_question = import_question_json()
    for term in data_question['questions']: # for all example
        for key,value in term.items(): # for all attribute of an example
            if key == 'answer':
                answer_list.append(value)
                answer_list.append('\n')
            if key == 'question':
                question_list.append(value)
                question_list.append('\n')
            if key == 'program': 
                for func_dic in value:
                    for key_, value_ in func_dic.items():
                        if key_ == 'function':
                            val = value_.lstrip('\"')+' '
                            program_list.append(val)
                program_list.append('. \n')
            else:
                pass
    with open("OutputData/answers.txt", "w+") as plfile:
        plfile.writelines(answer_list)
    plfile.close()       
    with open("OutputData/questions.txt", "w+") as plfile:
        plfile.writelines(question_list)
    plfile.close()        
    with open("OutputData/programs.txt", "w+") as plfile:
        plfile.writelines(program_list)
    plfile.close()

#generate_txt()

In [42]:
from itertools import permutations
from collections import Counter
import os

def main(x_list):
    sub_programs_dict = []
    sub_programs_str = ''
    try:
        os.mkdir('OutputData/program/')
    except:
        pass
    with open('OutputData/programs.txt') as f1:
        programs = f1.readlines()
        program_list = []
        for image_index in x_list:
            program = programs[image_index]
            program_list.append(program)
        with open('OutputData/program/'+str(len(x_list))+'_program.pl', "w") as plfile:
            print("{}".format(program_list), file=plfile)
        plfile.close()
    f1.close()
    #print(program_list)
    res = {}
    for pair_programs in permutations(program_list,2):
        (i,j) = pair_programs
        try:
            sub_program = two_program(i,j)
            sub_program = str(sub_program)
            num = sub_program.split(',')
            if len(num) > 3:
                if sub_program not in res.keys():
                    res[sub_program] = 0
                    sub_programs_dict.append(sub_program)
                    sub_programs_str += sub_program+'\n'
                else:
                    res[sub_program] += 1
            else:
                pass
        except:
            pass
    #sorted_res = sorted(res.keys(), key = lambda kv:(kv[1]),reverse=True)
    sorted_res = sorted(res.items(), key = lambda kv:(kv[1]),reverse=True)
    with open('OutputData/'+'sub_programs.txt','w') as f:
        for line in sorted_res:
            #print(line[0])
            #print(type(line[0]))
            f.writelines(str(line[0]))
            f.writelines('\n')
    f.close()

sub_pro_list = main(range(688,700))

In [11]:
def main_map(x_list):
    sub_programs_str = ''
    sub_programs_list = []
    try:
        os.mkdir('OutputData/program/')
    except:
        pass
    with open('OutputData/programs.txt') as f1:
        programs = f1.readlines()
        program_list = []
        for image_index in x_list:
            program = programs[image_index]
            program_list.append(program)
        with open('OutputData/program/'+str(len(x_list))+'_program.pl', "w") as plfile:
            print("{}".format(program_list), file=plfile)
        plfile.close()
    f1.close()
    #print(program_list)
    for program in program_list:
        try:
            #print(program)
            res = []
            edges = double_edges(program)
            for i,j in edges.items(): # i是边的信息,j是边的权
                if i[2] >= 1: #边是重边
                    double_edge = i[:2] # 重边，元组
                    res.append(double_edge)
                    res = list(set(res))
                else:
                    pass
            sub_programs_list.append(str(res))
            sub_programs_list.append('\n')
            sub_programs_str += str(res)+'\n'
        except:
            pass
    with open('OutputData/'+'edges.txt','w') as f:
        print("{}".format(sub_programs_str), file=f)
    f.close()
    return sub_programs_list

In [12]:
edges_map = main_map(range(60,700))
res = {}
for item in edges_map:
    graph = nx.Graph()
    num = item.split(',')
    if len(num) == 2:
        edge_nodes = item.replace(' ','').replace('[','').replace(']','').replace('(','').replace(')','').split(',')
        for i in range(len(edge_nodes)//2):
            graph.add_edges_from([(edge_nodes[2*i],edge_nodes[2*i+1])])
        try:
            nx.is_connected(graph)
            #print(item)        
            if item not in res.keys():   
                res[item] = 0
            else:
                res[item] += 1
        except:
            pass
top3 = sorted(res.items(), key = lambda kv:(kv[1]),reverse=True)[:100]
top3

[("[('unique', 'relate')]", 22),
 ("[('filter_size', 'filter_color')]", 11),
 ("[('filter_shape', 'unique')]", 11),
 ("[('scene', 'filter_color')]", 9),
 ("[('filter_color', 'filter_material')]", 8),
 ("[('scene', 'filter_size')]", 7),
 ("[('filter_material', 'filter_shape')]", 7),
 ("[('unique', 'query_shape')]", 5),
 ("[('filter_color', 'filter_shape')]", 5),
 ("[('filter_shape', 'count')]", 4),
 ("[('filter_size', 'filter_material')]", 3),
 ("[('unique', 'query_material')]", 3),
 ("[('filter_material', 'unique')]", 2),
 ("[('unique', 'query_color')]", 0)]